In [1]:
import json
from ast import literal_eval
import pandas as pd

In [2]:
projects = pd.read_csv("projects.csv")
users = pd.read_csv("user_names.csv")
afs = pd.read_csv("analysis_frameworks_2.csv")
regions = pd.read_csv("geo_regions.csv")

In [3]:
projects["stats_cache"] = projects["stats_cache"].apply(json.loads)
##
projects["number_of_entries"] = projects["stats_cache"].apply(
    lambda x: int(x["number_of_entries"]))
##
projects["number_of_users"] = projects["stats_cache"].apply(
    lambda x: int(x["number_of_users"]))
##
projects["number_of_leads_tagged"] = projects["stats_cache"].apply(
    lambda x: int(x["number_of_leads_tagged"]))
##
projects["number_of_leads_tagged_and_verified"] = projects[
    "stats_cache"].apply(
        lambda x: int(x["number_of_leads_tagged_and_verified"]))
##
projects = projects[projects["number_of_entries"] > 0]
##
projects["created_at"] = pd.to_datetime(
    projects["created_at"]).dt.tz_localize(None)
projects["latest_entry_date"] = pd.to_datetime(
    projects["latest_entry_date"]).dt.tz_localize(None)

In [4]:
regions["geo_options"] = regions["geo_options"].apply(json.loads)
def get_country(x):
    if x:
        for i in x:
            if i['label'].startswith("Country / "):
                return i['label'][len("Country / "):]
        return x[0]['label']
    else:
        return None
regions["country"] = regions["geo_options"].apply(get_country)

In [5]:
countries = regions[["project_id", "country"]].copy()
def countries_to_str(x):
    if x is None:
        return None
    final = []
    for i in x:
        if i is None:
            continue
        if i.endswith(" (cloned)"):
            i = i[:-len(" (cloned)")]
        final.append(i)
    return ', '.join([i for i in set(final) if i])
countries["countries"] = countries.groupby("project_id")['country'].transform(countries_to_str)
countries = countries.drop_duplicates(subset=["project_id", "countries"])[["project_id", "countries"]]

In [6]:
projects = pd.merge(
    projects,
    countries,
    left_on="id",
    right_on="project_id",
    suffixes=('', ''),
    how="left"
)

In [7]:
projects = projects[projects["title"].apply(lambda t: not("test" in t.lower() or "train" in t.lower()))]

In [8]:
projects = pd.merge(projects,
                    users,
                    how="left",
                    left_on="created_by_id",
                    right_on="id",
                    suffixes=("_project", "_user"))
projects["Created by"] = projects["first_name"] + " " + projects["last_name"]
projects = projects[[
    'title', 'modified_by_id', 'analysis_framework_id', 'description',
    'number_of_entries', "Created by", "organization", "created_at",
    "is_private", "status", 'number_of_users', 'number_of_leads_tagged',
       'number_of_leads_tagged_and_verified', "latest_entry_date", "countries"
]]

In [9]:
projects = pd.merge(projects,
                    users,
                    how="left",
                    left_on="modified_by_id",
                    right_on="id",
                    suffixes=("", "_user"))
projects["Modified by"] = projects["first_name"] + " " + projects["last_name"]
projects = projects[[
    'title', 'analysis_framework_id', 'description', 'number_of_entries',
    "Created by", "Modified by", "organization", "created_at", "is_private",
    "status", 'number_of_users', 'number_of_leads_tagged',
       'number_of_leads_tagged_and_verified', "latest_entry_date", "countries"
]]

In [10]:
projects = pd.merge(projects,
                    afs,
                    how="left",
                    left_on="analysis_framework_id",
                    right_on="id",
                    suffixes=("", "_af"))
projects["Analysis Framework Title"] = projects["title_af"]
projects = projects[[
    'title', 'number_of_entries', "Created by", "Modified by",
    "Analysis Framework Title", "organization", "created_at", "is_private",
    "status", "description", 'number_of_users', 'number_of_leads_tagged', "latest_entry_date", "countries"
]]

In [11]:
projects.sort_values("Analysis Framework Title", inplace=True)

In [12]:
projects.columns

Index(['title', 'number_of_entries', 'Created by', 'Modified by',
       'Analysis Framework Title', 'organization', 'created_at', 'is_private',
       'status', 'description', 'number_of_users', 'number_of_leads_tagged',
       'latest_entry_date', 'countries'],
      dtype='object')

In [13]:
projects.columns = [
    "Project Title", "Number of Entries", "Created by", "Modified by",
    "Analysis Framework Title", "Organization", 'Created at', 'Is Private?',
    'Status', 'Description', 'Number of Users', 'Number of Leads Tagged',
    "Latest Entry Date", "Geo Areas"
]
projects

,Project Title,Number of Entries,Created by,Modified by,Analysis Framework Title,Organization,Created at,Is Private?,Status,Description,Number of Users,Number of Leads Tagged,Latest Entry Date,Geo Areas
410,Nigeria Elections,38,Lauren Hargraves,Lauren Hargraves,(Test) Emergency Preparedness Framework,UNHCR,2019-02-27 10:59:05.875585,False,active,Analysis on Nigeria 2019 elections,1,10,2019-02-27 13:35:40.795081,"Nigeria, Ghana, Mauritania, Chad"
487,Balbasaur,68,Aditya Khatri,Safar Ligal,2020 Okular Analytics Framework,Togglecorp,2018-01-08 12:12:03.727026,False,active,Bulba bulba,13,7,2021-05-19 08:44:47.022857,"Colombia, Syrian Arab Republic, Nepal"
444,Cluster Viz,2,Aditya Khatri,Aditya Khatri,2020 Okular Analytics Framework,Togglecorp,2020-05-06 08:14:15.441216,False,active,NaN,25,2,2020-06-19 14:21:25.391577,"Ecuador, Colombia, Nepal"
443,2ndo proyecto de prueba Xavier,2,Xavier Lumen,Xavier Lumen,2020 Okular Analytics Framework,Okular,2020-06-05 07:27:10.263637,False,active,NaN,6,1,2020-06-05 13:31:28.947160,Spain
428,Board Demo,53,Ewan Oglethorpe,Ewan Oglethorpe,2020 Okular Analytics Framework,godeep,2018-03-16 05:28:11.444064,False,active,NaN,30,17,2020-10-01 13:52:06.275465,"t / , Kyrgyzstan, Tajikistan, Burundi, Saudi A..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,OCHA Burundi,87,Ana Maria Pereira,Ana Maria Pereira,_aa analysis introduction,United Nations Office for the Coordination of ...,2018-04-16 14:18:21.366568,False,active,NaN,6,22,2018-05-21 13:37:53.926522,Burundi
149,Prueba iMMAP Colombia,7,Alberto Castillo Aroca,Alberto Castillo Aroca,iMMAP Framework - Prueba,iMMAP,2020-01-13 15:29:42.555921,False,active,Una prueba para aprender DEEP,9,5,2020-06-08 20:30:19.254933,"Colombia, Venezuela"
94,SERF indicator,17,Faizal Thamrin,Faizal Thamrin,serf-indicators,InaSAFE (Indonesia),2020-09-03 03:07:01.254255,False,active,NaN,3,4,2020-11-03 03:02:07.768653,NaN
445,som_2042X,159,Elliott Messeiller,Elliott Messeiller,somalia_sdr,REACH Initiative (REACH Initiative),2020-05-04 09:21:56.043819,False,active,NaN,4,16,2020-05-14 14:59:30.618090,Somalia


In [14]:
with pd.ExcelWriter("Active Projects.xlsx", datetime_format='dd/mm/YY') as writer:
    projects[~projects["Is Private?"]].drop(columns=["Is Private?"]).to_excel(writer, index=None)

In [15]:
with pd.ExcelWriter("Private Projects.xlsx", datetime_format='dd/mm/YY') as writer:
    projects[projects["Is Private?"]].drop(columns=["Is Private?"]).to_excel(writer, index=None)

In [16]:
projects["Number of Entries"].sum()

283055